In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import matplotlib.pyplot as plt
import dask.dataframe as dd
import time
import statistics as st

In [2]:
#Replace with path + filename for your data
prescriptions_file = 'raw-med-data/opioid_prescription_mockid-BioVU-withGenericColumn_ER-2020-11-10.csv.gz'
icd_file = 'curated-opioid-data/icd-2005to2017-mockid.csv'
covariates_file = 'raw-med-data/20191120_sd_wide_medical_home_covariates_sex_race_median_age_density_records-mock_id-2020-08-17.txt'

In [3]:
df=pd.read_csv(prescriptions_file)
covariate= pd.read_csv(covariates_file)
icd = pd.read_csv(icd_file)

In [6]:
#include only medical_home=True (3+ years of data on record)
covariate=covariate[covariate['medical_home']]
#include only age>18
covariate=covariate[covariate['age']>=18]
#exclude patient with cancer icd code
cancer_icd=["140","140.0","140.1","140.3","140.4","140.5","140.6","140.8","140.9","141","141.0","141.1","141.2","141.3","141.4","141.5","141.6","141.8","141.9","142","142.0","142.1","142.2","142.8","142.9","143","143.0","143.1","143.8","143.9","144","144.0","144.1","144.8","144.9","145","145.0","145.1","145.2","145.3","145.4","145.5","145.6","145.8","145.9","146","146.0","146.1","146.2","146.3","146.4","146.5","146.6","146.7","146.8","146.9","147","147.0","147.1","147.2","147.3","147.8","147.9","148","148.0","148.1","148.2","148.3","148.8","148.9","149","149.0","149.1","149.8","149.9","150","150.0","150.1","150.2","150.3","150.4","150.5","150.8","150.9","151","151.0","151.1","151.2","151.3","151.4","151.5","151.6","151.8","151.9","152","152.0","152.1","152.2","152.3","152.8","152.9","153","153.0","153.1","153.2","153.3","153.4","153.5","153.6","153.7","153.8","153.9","154","154.0","154.1","154.2","154.3","154.8","155","155.0","155.1","155.2","156","156.0","156.1","156.2","156.8","156.9","157","157.0","157.1","157.2","157.3","157.4","157.8","157.9","158","158.0","158.8","158.9","159","159.0","159.1","159.8","159.9","160","160.0","160.1","160.2","160.3","160.4","160.5","160.8","160.9","161","161.0","161.1","161.2","161.3","161.8","161.9","162","162.0","162.2","162.3","162.4","162.5","162.8","162.9","163","163.0","163.1","163.8","163.9","164","164.0","164.1","164.2","164.3","164.8","164.9","165","165.0","165.8","165.9","170","170.0","170.1","170.2","170.3","170.4","170.5","170.6","170.7","170.8","170.9","171","171.0","171.2","171.3","171.4","171.5","171.6","171.7","171.8","171.9","172","172.0","172.1","172.2","172.3","172.4","172.5","172.6","172.7","172.8","172.9","173","173.0","173.00","173.01","173.02","173.09","173.1","173.10","173.11","173.12","173.19","173.2","173.20","173.21","173.22","173.29","173.3","173.30","173.31","173.32","173.39","173.4","173.40","173.41","173.42","173.49","173.5","173.50","173.51","173.52","173.59","173.6","173.60","173.61","173.62","173.69","173.7","173.70","173.71","173.72","173.79","173.8","173.80","173.81","173.82","173.89","173.9","173.90","173.91","173.92","173.99","174","174.0","174.1","174.2","174.3","174.4","174.5","174.6","174.8","174.9","175","175.0","175.9","176","176.0","176.1","176.2","176.3","176.4","176.5","176.8","176.9","179","180","180.0","180.1","180.8","180.9","181","182","182.0","182.1","182.8","183","183.0","183.2","183.3","183.4","183.5","183.8","183.9","184","184.0","184.1","184.2","184.3","184.4","184.8","184.9","185","186","186.0","186.9","187","187.1","187.2","187.3","187.4","187.5","187.6","187.7","187.8","187.9","188","188.0","188.1","188.2","188.3","188.4","188.5","188.6","188.7","188.8","188.9","189","189.0","189.1","189.2","189.3","189.4","189.8","189.9","190","190.0","190.1","190.2","190.3","190.4","190.5","190.6","190.7","190.8","190.9","191","191.0","191.1","191.2","191.3","191.4","191.5","191.6","191.7","191.8","191.9","192","192.0","192.1","192.2","192.3","192.8","192.9","193","194","194.0","194.1","194.3","194.4","194.5","194.6","194.8","194.9","195","195.0","195.1","195.2","195.3","195.4","195.5","195.8","196","196.0","196.1","196.2","196.3","196.5","196.6","196.8","196.9","197","197.0","197.1","197.2","197.3","197.4","197.5","197.6","197.7","197.8","198","198.0","198.1","198.2","198.3","198.4","198.5","198.6","198.7","198.8","198.81","198.82","198.89","199","199.0","199.1","199.2","200","200.0","200.00","200.01","200.02","200.03","200.04","200.05","200.06","200.07","200.08","200.1","200.10","200.11","200.12","200.13","200.14","200.15","200.16","200.17","200.18","200.2","200.20","200.21","200.22","200.23","200.24","200.25","200.26","200.27","200.28","200.3","200.30","200.31","200.32","200.33","200.34","200.35","200.36","200.37","200.38","200.4","200.40","200.41","200.42","200.43","200.44","200.45","200.46","200.47","200.48","200.5","200.50","200.51","200.52","200.53","200.54","200.55","200.56","200.57","200.58","200.6","200.60","200.61","200.62","200.63","200.64","200.65","200.66","200.67","200.68","200.7","200.70","200.71","200.72","200.73","200.74","200.75","200.76","200.77","200.78","200.8","200.80","200.81","200.82","200.83","200.84","200.85","200.86","200.87","200.88","200000","201","201.0","201.00","201.01","201.02","201.03","201.04","201.05","201.06","201.07","201.08","201.1","201.10","201.11","201.12","201.13","201.14","201.15","201.16","201.17","201.18","201.2","201.20","201.21","201.22","201.23","201.24","201.25","201.26","201.27","201.28","201.4","201.40","201.41","201.42","201.43","201.44","201.45","201.46","201.47","201.48","201.5","201.50","201.51","201.52","201.53","201.54","201.55","201.56","201.57","201.58","201.6","201.60","201.61","201.62","201.63","201.64","201.65","201.66","201.67","201.68","201.7","201.70","201.71","201.72","201.73","201.74","201.75","201.76","201.77","201.78","201.9","201.90","201.91","201.92","201.93","201.94","201.95","201.96","201.97","201.98","202","202.0","202.00","202.01","202.02","202.03","202.04","202.05","202.06","202.07","202.08","202.1","202.10","202.11","202.12","202.13","202.14","202.15","202.16","202.17","202.18","202.2","202.20","202.21","202.22","202.23","202.24","202.25","202.26","202.27","202.28","202.3","202.30","202.31","202.32","202.33","202.34","202.35","202.36","202.37","202.38","202.4","202.40","202.41","202.42","202.43","202.44","202.45","202.46","202.47","202.48","202.5","202.50","202.51","202.52","202.53","202.54","202.55","202.56","202.57","202.58","202.6","202.60","202.61","202.62","202.63","202.64","202.65","202.66","202.67","202.68","202.7","202.70","202.71","202.72","202.73","202.74","202.75","202.76","202.77","202.78","202.8","202.80","202.81","202.82","202.83","202.84","202.85","202.86","202.87","202.88","202.9","202.90","202.91","202.92","202.93","202.94","202.95","202.96","202.97","202.98","203","203.0","203.00","203.01","203.02","203.1","203.10","203.11","203.12","203.8","203.80","203.81","203.82","204","204.0","204.00","204.01","204.02","204.1","204.10","204.11","204.12","204.2","204.20","204.21","204.22","204.8","204.80","204.81","204.82","204.9","204.90","204.91","204.92","205","205.0","205.00","205.01","205.02","205.1","205.10","205.11","205.12","205.2","205.20","205.21","205.22","205.3","205.30","205.31","205.32","205.8","205.80","205.81","205.82","205.9","205.90","205.91","205.92","206","206.0","206.00","206.01","206.02","206.1","206.10","206.11","206.12","206.2","206.20","206.21","206.22","206.8","206.80","206.81","206.82","206.9","206.90","206.91","206.92","207","207.0","207.00","207.01","207.02","207.1","207.10","207.11","207.12","207.2","207.20","207.21","207.22","207.8","207.80","207.81","207.82","208","208.0","208.00","208.01","208.02","208.1","208.10","208.11","208.12","208.2","208.20","208.21","208.22","208.8","208.80","208.81","208.82","208.9","208.90","208.91","208.92","209.0","209.00","209.01","209.02","209.03","209.1","209.10","209.11","209.12","209.13","209.14","209.15","209.16","209.17","209.2","209.20","209.21","209.22","209.23","209.24","209.25","209.26","209.27","209.29","209.3","209.30","209.31","209.32","209.33","209.34","209.35","209.36","C00","C00.0","C00.1","C00.2","C00.3","C00.4","C00.5","C00.6","C00.8","C00.9","C01","C02","C02.0","C02.1","C02.2","C02.3","C02.4","C02.8","C02.9","C03","C03.0","C03.1","C03.9","C04","C04.0","C04.1","C04.8","C04.9","C05","C05.0","C05.1","C05.2","C05.8","C05.9","C06","C06.0","C06.1","C06.2","C06.8","C06.80","C06.89","C06.9","C07","C08","C08.0","C08.1","C08.9","C09","C09.0","C09.1","C09.8","C09.9","C10","C10.0","C10.1","C10.2","C10.3","C10.4","C10.8","C10.9","C11","C11.0","C11.1","C11.2","C11.3","C11.8","C11.9","C12","C13","C13.0","C13.1","C13.2","C13.8","C13.9","C14","C14.0","C14.2","C14.8","C15","C15.3","C15.4","C15.5","C15.8","C15.9","C16","C16.0","C16.1","C16.2","C16.3","C16.4","C16.5","C16.6","C16.8","C16.9","C17","C17.0","C17.1","C17.2","C17.3","C17.8","C17.9","C18","C18.0","C18.1","C18.2","C18.3","C18.4","C18.5","C18.6","C18.7","C18.8","C18.9","C19","C20","C21","C21.0","C21.1","C21.2","C21.8","C22","C22.0","C22.1","C22.2","C22.3","C22.4","C22.7","C22.8","C22.9","C23","C24","C24.0","C24.1","C24.8","C24.9","C25","C25.0","C25.1","C25.2","C25.3","C25.4","C25.7","C25.8","C25.9","C26","C26.0","C26.1","C26.9","C30","C30.0","C30.1","C31","C31.0","C31.1","C31.2","C31.3","C31.8","C31.9","C32","C32.0","C32.1","C32.2","C32.3","C32.8","C32.9","C33","C34","C34.0","C34.00","C34.01","C34.02","C34.1","C34.10","C34.11","C34.12","C34.2","C34.3","C34.30","C34.31","C34.32","C34.8","C34.80","C34.81","C34.82","C34.9","C34.90","C34.91","C34.92","C37","C38","C38.0","C38.1","C38.2","C38.3","C38.4","C38.8","C39","C39.0","C39.9","C40","C40.0","C40.00","C40.01","C40.02","C40.1","C40.10","C40.11","C40.12","C40.2","C40.20","C40.21","C40.22","C40.3","C40.30","C40.31","C40.32","C40.8","C40.80","C40.81","C40.82","C40.9","C40.90","C40.91","C40.92","C41","C41.0","C41.1","C41.2","C41.3","C41.4","C41.9","C43","C43.0","C43.1","C43.10","C43.11","C43.111","C43.112","C43.12","C43.121","C43.122","C43.2","C43.20","C43.21","C43.22","C43.3","C43.30","C43.31","C43.39","C43.4","C43.5","C43.51","C43.52","C43.59","C43.6","C43.60","C43.61","C43.62","C43.7","C43.70","C43.71","C43.72","C43.8","C43.9","C44","C44.0","C44.00","C44.01","C44.02","C44.09","C44.1","C44.10","C44.101","C44.102","C44.1021","C44.1022","C44.109","C44.1091","C44.1092","C44.11","C44.111","C44.112","C44.1121","C44.1122","C44.119","C44.1191","C44.1192","C44.12","C44.121","C44.122","C44.1221","C44.1222","C44.129","C44.1291","C44.1292","C44.131","C44.1321","C44.1322","C44.1391","C44.1392","C44.19","C44.191","C44.192","C44.1921","C44.1922","C44.199","C44.1991","C44.1992","C44.2","C44.20","C44.201","C44.202","C44.209","C44.21","C44.211","C44.212","C44.219","C44.22","C44.221","C44.222","C44.229","C44.29","C44.291","C44.292","C44.299","C44.3","C44.30","C44.300","C44.301","C44.309","C44.31","C44.310","C44.311","C44.319","C44.32","C44.320","C44.321","C44.329","C44.39","C44.390","C44.391","C44.399","C44.4","C44.40","C44.41","C44.42","C44.49","C44.5","C44.50","C44.500","C44.501","C44.509","C44.51","C44.510","C44.511","C44.519","C44.52","C44.520","C44.521","C44.529","C44.59","C44.590","C44.591","C44.599","C44.6","C44.60","C44.601","C44.602","C44.609","C44.61","C44.611","C44.612","C44.619","C44.62","C44.621","C44.622","C44.629","C44.69","C44.691","C44.692","C44.699","C44.7","C44.70","C44.701","C44.702","C44.709","C44.71","C44.711","C44.712","C44.719","C44.72","C44.721","C44.722","C44.729","C44.79","C44.791","C44.792","C44.799","C44.8","C44.80","C44.81","C44.82","C44.89","C44.9","C44.90","C44.91","C44.92","C44.99","C45","C45.0","C45.1","C45.2","C45.7","C45.9","C46","C46.0","C46.1","C46.2","C46.3","C46.4","C46.5","C46.50","C46.51","C46.52","C46.7","C46.9","C47","C47.0","C47.1","C47.10","C47.11","C47.12","C47.2","C47.20","C47.21","C47.22","C47.3","C47.4","C47.5","C47.6","C47.8","C47.9","C48","C48.0","C48.1","C48.2","C48.8","C49","C49.0","C49.1","C49.10","C49.11","C49.12","C49.2","C49.20","C49.21","C49.22","C49.3","C49.4","C49.5","C49.6","C49.8","C49.9","C49.A","C49.A0","C49.A1","C49.A2","C49.A3","C49.A4","C49.A5","C49.A9","C4A","C4A.0","C4A.1","C4A.10","C4A.11","C4A.111","C4A.112","C4A.12","C4A.121","C4A.122","C4A.2","C4A.20","C4A.21","C4A.22","C4A.3","C4A.30","C4A.31","C4A.39","C4A.4","C4A.5","C4A.51","C4A.52","C4A.59","C4A.6","C4A.60","C4A.61","C4A.62","C4A.7","C4A.70","C4A.71","C4A.72","C4A.8","C4A.9","C50","C50.0","C50.01","C50.011","C50.012","C50.019","C50.02","C50.021","C50.022","C50.029","C50.1","C50.11","C50.111","C50.112","C50.119","C50.12","C50.121","C50.122","C50.129","C50.2","C50.21","C50.211","C50.212","C50.219","C50.22","C50.221","C50.222","C50.229","C50.3","C50.31","C50.311","C50.312","C50.319","C50.32","C50.321","C50.322","C50.329","C50.4","C50.41","C50.411","C50.412","C50.419","C50.42","C50.421","C50.422","C50.429","C50.5","C50.51","C50.511","C50.512","C50.519","C50.52","C50.521","C50.522","C50.529","C50.6","C50.61","C50.611","C50.612","C50.619","C50.62","C50.621","C50.622","C50.629","C50.8","C50.81","C50.811","C50.812","C50.819","C50.82","C50.821","C50.822","C50.829","C50.9","C50.91","C50.911","C50.912","C50.919","C50.92","C50.921","C50.922","C50.929","C51","C51.0","C51.1","C51.2","C51.8","C51.9","C52","C53","C53.0","C53.1","C53.8","C53.9","C54","C54.0","C54.1","C54.2","C54.3","C54.8","C54.9","C55","C56","C56.0","C56.1","C56.2","C56.9","C57","C57.0","C57.00","C57.01","C57.02","C57.1","C57.10","C57.11","C57.12","C57.2","C57.20","C57.21","C57.22","C57.3","C57.4","C57.7","C57.8","C57.9","C58","C60","C60.0","C60.1","C60.2","C60.8","C60.9","C61","C62","C62.0","C62.00","C62.01","C62.02","C62.1","C62.10","C62.11","C62.12","C62.9","C62.90","C62.91","C62.92","C63","C63.0","C63.00","C63.01","C63.02","C63.1","C63.10","C63.11","C63.12","C63.2","C63.7","C63.8","C63.9","C64","C64.0","C64.1","C64.2","C64.9","C65","C65.0","C65.1","C65.2","C65.9","C66","C66.0","C66.1","C66.2","C66.9","C67","C67.0","C67.1","C67.2","C67.3","C67.4","C67.5","C67.6","C67.7","C67.8","C67.9","C68","C68.0","C68.1","C68.8","C68.9","C69","C69.0","C69.00","C69.01","C69.02","C69.1","C69.10","C69.11","C69.12","C69.2","C69.20","C69.21","C69.22","C69.3","C69.30","C69.31","C69.32","C69.4","C69.40","C69.41","C69.42","C69.5","C69.50","C69.51","C69.52","C69.6","C69.60","C69.61","C69.62","C69.8","C69.80","C69.81","C69.82","C69.9","C69.90","C69.91","C69.92","C70","C70.0","C70.1","C70.9","C71","C71.0","C71.1","C71.2","C71.3","C71.4","C71.5","C71.6","C71.7","C71.8","C71.9","C72","C72.0","C72.1","C72.2","C72.20","C72.21","C72.22","C72.3","C72.30","C72.31","C72.32","C72.4","C72.40","C72.41","C72.42","C72.5","C72.50","C72.59","C72.9","C73","C74","C74.0","C74.00","C74.01","C74.02","C74.1","C74.10","C74.11","C74.12","C74.9","C74.90","C74.91","C74.92","C75","C75.0","C75.1","C75.2","C75.3","C75.4","C75.5","C75.8","C75.9","C76","C76.0","C76.1","C76.2","C76.3","C76.4","C76.40","C76.41","C76.42","C76.5","C76.50","C76.51","C76.52","C76.7","C76.8","C77","C77.0","C77.1","C77.2","C77.3","C77.4","C77.5","C77.8","C77.9","C78","C78.0","C78.00","C78.01","C78.02","C78.1","C78.2","C78.3","C78.30","C78.39","C78.4","C78.5","C78.6","C78.7","C78.8","C78.80","C78.89","C79","C79.0","C79.00","C79.01","C79.02","C79.1","C79.10","C79.11","C79.19","C79.2","C79.3","C79.31","C79.32","C79.4","C79.40","C79.49","C79.5","C79.51","C79.52","C79.6","C79.60","C79.61","C79.62","C79.7","C79.70","C79.71","C79.72","C79.8","C79.81","C79.82","C79.89","C79.9","C7A","C7A.0","C7A.00","C7A.01","C7A.010","C7A.011","C7A.012","C7A.019","C7A.02","C7A.020","C7A.021","C7A.022","C7A.023","C7A.024","C7A.025","C7A.026","C7A.029","C7A.09","C7A.090","C7A.091","C7A.092","C7A.093","C7A.094","C7A.095","C7A.096","C7A.098","C7A.1","C7A.8","C7B","C7B.0","C7B.00","C7B.01","C7B.02","C7B.03","C7B.04","C7B.09","C7B.1","C7B.8","C80","C80.0","C80.1","C80.2","C81","C81.0","C81.00","C81.01","C81.02","C81.03","C81.04","C81.05","C81.06","C81.07","C81.08","C81.09","C81.1","C81.10","C81.11","C81.12","C81.13","C81.14","C81.15","C81.16","C81.17","C81.18","C81.19","C81.2","C81.20","C81.21","C81.22","C81.23","C81.24","C81.25","C81.26","C81.27","C81.28","C81.29","C81.3","C81.30","C81.31","C81.32","C81.33","C81.34","C81.35","C81.36","C81.37","C81.38","C81.39","C81.4","C81.40","C81.41","C81.42","C81.43","C81.44","C81.45","C81.46","C81.47","C81.48","C81.49","C81.7","C81.70","C81.71","C81.72","C81.73","C81.74","C81.75","C81.76","C81.77","C81.78","C81.79","C81.9","C81.90","C81.91","C81.92","C81.93","C81.94","C81.95","C81.96","C81.97","C81.98","C81.99","C82","C82.0","C82.00","C82.01","C82.02","C82.03","C82.04","C82.05","C82.06","C82.07","C82.08","C82.09","C82.1","C82.10","C82.11","C82.12","C82.13","C82.14","C82.15","C82.16","C82.17","C82.18","C82.19","C82.2","C82.20","C82.21","C82.22","C82.23","C82.24","C82.25","C82.26","C82.27","C82.28","C82.29","C82.3","C82.30","C82.31","C82.32","C82.33","C82.34","C82.35","C82.36","C82.37","C82.38","C82.39","C82.4","C82.40","C82.41","C82.42","C82.43","C82.44","C82.45","C82.46","C82.47","C82.48","C82.49","C82.5","C82.50","C82.51","C82.52","C82.53","C82.54","C82.55","C82.56","C82.57","C82.58","C82.59","C82.6","C82.60","C82.61","C82.62","C82.63","C82.64","C82.65","C82.66","C82.67","C82.68","C82.69","C82.70","C82.71","C82.72","C82.73","C82.74","C82.75","C82.76","C82.77","C82.78","C82.79","C82.8","C82.80","C82.81","C82.82","C82.83","C82.84","C82.85","C82.86","C82.87","C82.88","C82.89","C82.9","C82.90","C82.91","C82.92","C82.93","C82.94","C82.95","C82.96","C82.97","C82.98","C82.99","C83","C83.0","C83.00","C83.01","C83.02","C83.03","C83.04","C83.05","C83.06","C83.07","C83.08","C83.09","C83.1","C83.10","C83.11","C83.12","C83.13","C83.14","C83.15","C83.16","C83.17","C83.18","C83.19","C83.20","C83.21","C83.22","C83.23","C83.24","C83.25","C83.26","C83.27","C83.28","C83.29","C83.3","C83.30","C83.31","C83.32","C83.33","C83.34","C83.35","C83.36","C83.37","C83.38","C83.39","C83.40","C83.41","C83.42","C83.43","C83.44","C83.45","C83.46","C83.47","C83.48","C83.49","C83.5","C83.50","C83.51","C83.52","C83.53","C83.54","C83.55","C83.56","C83.57","C83.58","C83.59","C83.60","C83.61","C83.62","C83.63","C83.64","C83.65","C83.66","C83.67","C83.68","C83.69","C83.7","C83.70","C83.71","C83.72","C83.73","C83.74","C83.75","C83.76","C83.77","C83.78","C83.79","C83.8","C83.80","C83.81","C83.82","C83.83","C83.84","C83.85","C83.86","C83.87","C83.88","C83.89","C83.9","C83.90","C83.91","C83.92","C83.93","C83.94","C83.95","C83.96","C83.97","C83.98","C83.99","C84","C84.0","C84.00","C84.01","C84.02","C84.03","C84.04","C84.05","C84.06","C84.07","C84.08","C84.09","C84.1","C84.10","C84.11","C84.12","C84.13","C84.14","C84.15","C84.16","C84.17","C84.18","C84.19","C84.20","C84.21","C84.22","C84.23","C84.24","C84.25","C84.26","C84.27","C84.28","C84.29","C84.30","C84.31","C84.32","C84.33","C84.34","C84.35","C84.36","C84.37","C84.38","C84.39","C84.4","C84.40","C84.41","C84.42","C84.43","C84.44","C84.45","C84.46","C84.47","C84.48","C84.49","C84.50","C84.51","C84.52","C84.53","C84.54","C84.55","C84.56","C84.57","C84.58","C84.59","C84.6","C84.60","C84.61","C84.62","C84.63","C84.64","C84.65","C84.66","C84.67","C84.68","C84.69","C84.7","C84.70","C84.71","C84.72","C84.73","C84.74","C84.75","C84.76","C84.77","C84.78","C84.79","C84.9","C84.90","C84.91","C84.92","C84.93","C84.94","C84.95","C84.96","C84.97","C84.98","C84.99","C84.A","C84.A0","C84.A1","C84.A2","C84.A3","C84.A4","C84.A5","C84.A6","C84.A7","C84.A8","C84.A9","C84.Z","C84.Z0","C84.Z1","C84.Z2","C84.Z3","C84.Z4","C84.Z5","C84.Z6","C84.Z7","C84.Z8","C84.Z9","C85","C85.00","C85.01","C85.02","C85.03","C85.04","C85.05","C85.06","C85.07","C85.08","C85.09","C85.1","C85.10","C85.11","C85.12","C85.13","C85.14","C85.15","C85.16","C85.17","C85.18","C85.19","C85.2","C85.20","C85.21","C85.22","C85.23","C85.24","C85.25","C85.26","C85.27","C85.28","C85.29","C85.70","C85.71","C85.72","C85.73","C85.74","C85.75","C85.76","C85.77","C85.78","C85.79","C85.8","C85.80","C85.81","C85.82","C85.83","C85.84","C85.85","C85.86","C85.87","C85.88","C85.89","C85.9","C85.90","C85.91","C85.92","C85.93","C85.94","C85.95","C85.96","C85.97","C85.98","C85.99","C86","C86.0","C86.1","C86.2","C86.3","C86.4","C86.5","C86.6","C88","C88.0","C88.1","C88.2","C88.3","C88.4","C88.7","C88.8","C88.9","C90","C90.0","C90.00","C90.01","C90.02","C90.1","C90.10","C90.11","C90.12","C90.2","C90.20","C90.21","C90.22","C90.3","C90.30","C90.31","C90.32","C91","C91.0","C91.00","C91.01","C91.02","C91.1","C91.10","C91.11","C91.12","C91.20","C91.21","C91.3","C91.30","C91.31","C91.32","C91.4","C91.40","C91.41","C91.42","C91.5","C91.50","C91.51","C91.52","C91.6","C91.60","C91.61","C91.62","C91.70","C91.71","C91.9","C91.90","C91.91","C91.92","C91.A","C91.A0","C91.A1","C91.A2","C91.Z","C91.Z0","C91.Z1","C91.Z2","C92","C92.0","C92.00","C92.01","C92.02","C92.1","C92.10","C92.11","C92.12","C92.2","C92.20","C92.21","C92.22","C92.3","C92.30","C92.31","C92.32","C92.4","C92.40","C92.41","C92.42","C92.5","C92.50","C92.51","C92.52","C92.6","C92.60","C92.61","C92.62","C92.70","C92.71","C92.9","C92.90","C92.91","C92.92","C92.A","C92.A0","C92.A1","C92.A2","C92.Z","C92.Z0","C92.Z1","C92.Z2","C93","C93.0","C93.00","C93.01","C93.02","C93.1","C93.10","C93.11","C93.12","C93.20","C93.21","C93.3","C93.30","C93.31","C93.32","C93.70","C93.71","C93.9","C93.90","C93.91","C93.92","C93.Z","C93.Z0","C93.Z1","C93.Z2","C94","C94.0","C94.00","C94.01","C94.02","C94.10","C94.11","C94.2","C94.20","C94.21","C94.22","C94.3","C94.30","C94.31","C94.32","C94.4","C94.40","C94.41","C94.42","C94.50","C94.51","C94.6","C94.70","C94.71","C94.8","C94.80","C94.81","C94.82","C95","C95.0","C95.00","C95.01","C95.02","C95.1","C95.10","C95.11","C95.12","C95.20","C95.21","C95.70","C95.71","C95.9","C95.90","C95.91","C95.92","C96","C96.0","C96.1","C96.2","C96.20","C96.21","C96.22","C96.29","C96.3","C96.4","C96.5","C96.6","C96.7","C96.9","C96.A","C96.Z"]
cancer_ls= icd[icd['ICD_CODE'].isin(cancer_icd)]['GRID']
covariate=covariate[~covariate['GRID'].isin(cancer_ls)]



In [7]:
icd=icd[icd['GRID'].isin(covariate['GRID'])]
df=df[df['GRID'].isin(covariate['GRID'])]

In [27]:
#searchfor = ['syringe', 'inject']
#df_injectables=df[df['MED_NAME'].str.contains('|'.join(searchfor))]


In [9]:
df['MED_DATE']=pd.to_datetime(df['MED_DATE'])
covariate['DOB']= pd.to_datetime(covariate['DOB'])



In [10]:
#n: how many prescription in each definition
#days:  how many days between prescription

def grid_by_time(dates,n,days,chronic1=False,exclude_one_day_apart=False,expose=False,bout=False):
    #remove same dates
    dates=dates.apply(lambda x :sorted(set(x)))
    #remove grid with less than n prescriptions
    dates=dates[dates.apply(lambda x : False if len(x)<n else True)]
    def func_bout(ls):
        start=np.nan
        end=np.nan
        bout_ls=[]
        for i in range(len(ls)-1):
            if ((ls[i+1]-ls[i])<=dt.timedelta(days)): 
                if type(start)!=pd.Timestamp:
                    if not exclude_one_day_apart:
                        start=ls[i]
                    elif ((ls[i+1]-ls[i])>dt.timedelta(1)):
                        start=ls[i]
            elif type(start)==pd.Timestamp:
                end=ls[i]
                bout_ls.append(end-start)
                start=np.nan
        if type(start)==pd.Timestamp:
            bout_ls.append(ls[-1]-start)
        return bout_ls                
            
    def func_expose(ls):
        for i in range(len(ls)-1)[1:]:
            if ((ls[i+1]-ls[i])<=dt.timedelta(days)) & ((ls[i]-ls[i-1])>=dt.timedelta(270)): 
                return True
            elif i==1:
                if ((ls[i]-ls[0])<=dt.timedelta(days)) &  ((ls[i+1]-ls[i])>=dt.timedelta(270)):
                    return True
        return False
    def func_chronic1 (ls): 
        count=0
        for i in range(len(ls)-1):
            if count==n-1:
                return True
            elif ((ls[i+1]-ls[i])<=dt.timedelta(days)): 
                if not exclude_one_day_apart:
                    count+=1
                elif ((ls[i+1]-ls[i])>dt.timedelta(1)):
                    count+=1
            else:
                count=0
        if count==2:
            return True
        else:
            return False
    def func_chronic2(ls):
        for i in range(len(ls)-(n-1)):
            if (ls[i+9]-ls[i])<=dt.timedelta(days):
                if not exclude_one_day_apart:
                    return True
                else:
                    sub_ls= ls[i:i+n]
                    count=0
                    for j in range(len(sub_ls)-1):
                        if ((sub_ls[j+1]-sub_ls[j])<=dt.timedelta(1)):
                            break;
                        else:
                            count+=1
                    if count == 9:
                        return True
        return False
    
    if chronic1:
        return dates.index[dates.apply(func_chronic1)].tolist()
    elif expose:
        return dates.index[dates.apply(func_expose)].tolist()
    elif bout:
        return dates.apply(func_bout)
    else:
        return dates.index[dates.apply(func_chronic2)].tolist()        

In [11]:
#Check if ICD codes on same day
oud_icd = ["304.0","304.01","304.02","304.03","304.7","304.70","304.71","304.72","304.8","305.5","305.50","305.51","305.52","305.53","F11.29","F11.10","F11.120","F11.121","F11.122","F11.129","F11.14","F11.150","F11.151","F11.159","F11.181","F11.182","F11.188","F11.19","F11.20","F11.21","F11.220","F11.221","F11.222","F11.229","F11.23","F11.24","F11.250","F11.251","F11.259","F11.281","F11.282","F11.288"]
icd['ICD_DATE']=pd.to_datetime(icd['ICD_DATE'])
icd_oud=icd[icd['ICD_CODE'].isin(oud_icd)]

dates_icd= icd_oud.groupby('GRID')['ICD_DATE'].apply(sorted)

def func_same_day_oud_icd (ls): 
        count=0
        for i in range(len(ls)-1):
            if ((ls[i+1]-ls[i])==dt.timedelta(0)): 
                    count+=1
        if count>0:
            return True
        else:
            return False


grids_with_same_day_oud_icd = dates_icd.index[dates_icd.apply(func_same_day_oud_icd)].tolist()




In [12]:
dates= df.groupby('GRID')['MED_DATE'].apply(sorted)

#To exclude from exposed control group:
chronic1_ls=grid_by_time(dates,n=3,days=90,chronic1=True,exclude_one_day_apart=True)

#Calculate bouts
bout = grid_by_time(dates,2,90,bout=True,exclude_one_day_apart=False)

In [14]:
oud_icd = ["304.0","304.01","304.02","304.03","304.7","304.70","304.71","304.72","305.5","305.50","305.51","305.52","305.53","F11.29","F11.10","F11.120","F11.121","F11.122","F11.129","F11.14","F11.150","F11.151","F11.159","F11.181","F11.182","F11.188","F11.19","F11.20","F11.21","F11.220","F11.221","F11.222","F11.229","F11.23","F11.24","F11.250","F11.251","F11.259","F11.281","F11.282","F11.288"]

#Everyone
everyone_ls=covariate['GRID'].to_list()

#No prescriptions
no_prescription_ls=covariate[~covariate['GRID'].isin(df['GRID'])]['GRID'].to_list()

#Exposed controls
exposed_ls=grid_by_time(dates,2,90,chronic1=False,exclude_one_day_apart=False,expose=True)
exposed_ls= list(set(exposed_ls)-set(chronic1_ls))

#Chronic
chronic2_ls=grid_by_time(dates,10,365,chronic1=False,exclude_one_day_apart=True)

###OUD from 1 ICD code
oud1_ls=icd[icd['ICD_CODE'].isin(oud_icd)]['GRID'].unique()





In [15]:
#Select ICD codes

#the function takes in 3 parameters: ls is the list of icd codes that we want to select,grid_ls is a list of people we care about, n is the number of occurrences of the icd code required
#the function returns a list of unique GRIDs who have the icd codes in ls

def select_icd(ls,grid_ls,n=1):
    if n==1:
        return set(icd[icd['ICD_CODE'].isin(ls)]['GRID']) &set(grid_ls)
    else:
        grid = icd[icd['ICD_CODE'].isin(ls)].groupby('GRID')['ICD_DATE'].count()
        ls = grid.index[grid>1].tolist()
        del grid
        return set(ls) &set(grid_ls)




In [16]:
oud_icd = ["304.0","304.01","304.02","304.03","304.7","304.70","304.71","304.72","304.8","305.5","305.50","305.51","305.52","305.53","F11.29","F11.10","F11.120","F11.121","F11.122","F11.129","F11.14","F11.150","F11.151","F11.159","F11.181","F11.182","F11.188","F11.19","F11.20","F11.21","F11.220","F11.221","F11.222","F11.229","F11.23","F11.24","F11.250","F11.251","F11.259","F11.281","F11.282","F11.288"]
Z79899_icd = ["Z79.899"]
aud_icd=["291","291.3","291.4","291.81","291.89","291.9","303","303","303.01","303.01","303.02","303.03","303.9","303.91","303.92","303.92","303.93","305","305.01","305.02","305.03"," 291.82"," 305.0","291.0","291.1","291.2","291.5","291.8","291.81","303.0","303.00","303.02","303.03","303.90","305.00","305.00","357.5","425.5","535.3","535.30","535.31","571.0","571.1","571.2","571.3","790.3","94.46","94.53","94.6","94.61","94.62","94.63","94.67","94.68","94.69","977.3","980","980.0","980.1","980.2","980.8","980.9","E947.3","V11.3","F10.121","F10.231","F10.251","F10.929","F10.230","F10.239","F10.159","F10.288","F10.19","F10.29","F10.99","F10.229","F10.20","F10.21","F10.10","F10.11","F10.220","F10.120","F10.129","E24.4","F10.14","F10.150","F10.151","F10.180","F10.181","F10.182","F10.188","F10.221","F10.232","F10.24","F10.250","F10.259","F10.26","F10.27","F10.280","F10.281","F10.282","F10.920","F10.921","F10.94","F10.950","F10.951","F10.959","F10.96","F10.97","F10.980","F10.981","F10.982","F10.988","G31.2","G62.1","G72.1","I42.6","K29.20","K29.21","K70.0","K70.10","K70.11","K70.2","K70.30","K70.31","K70.40","K70.41","K70.9","K85.20","K85.21","K85.22","K86.0","O35.4XX0","O35.4XX1","O35.4XX2","O35.4XX3","O35.4XX4","O35.4XX5","O35.4XX9","O99.310","O99.311","O99.312","O99.313","O99.314","O99.315","T51.8X2A","T51.8X2D","T51.8X2S","T51.92XA","T51.92XD","T51.92XS","Y90.5"," Y90.6","Y90.7","Y90.8","Z71.41"]
tud_icd= ["292.89","292.9","305.1","305.1","305.11","305.12","305.13","649.0","649.01","649.02","649.03","649.04","989.84","F17.218","F17.228","F17.209","F17.219","F17.229","F17.200","F17.201","F17.210","F17.220","F17.290","F17.21","V15.82","F17.220","F17.221","F17.223","F17.228","F17.229","F17.290","F17.291","F17.293","F17.298","F17.299","O99.330","O99.331","O99.332","O99.333","O99.334","O99.335","Z71.6"]
cud_icd=["304.3", "304.30", "304.31", "304.32", "304.33", "305.2", "305.20", "305.21", "305.22", "305.23", "F12", "F12.1", "F12.10", "F12.120", "F12.121", "F12.122", "F12.129", "F12.150", "F12.151", "F12.159", "F12.180", "F12.188", "F12.19", "F12.2", "F12.20", "F12.21", "F12.220", "F12.221", "F12.222", "F12.229", "F12.250", "F12.251", "F12.259", "F12.280", "F12.288", "F12.29", "F12.90", "F12.920", "F12.921", "F12.922", "F12.929", "F12.950", "F12.951", "F12.959", "F12.980", "F12.988", "F12.99"]
other_sud_icd = ["292","292.0","292.11","292.12","292.2","292.84","292.85","292.89","292.9","304.1","304.11","304.13","304.2","304.23","304.3","304.31","304.32","304.33","304.4","304.40","304.41","304.42","304.43","304.5","304.6","304.61","304.63","304.7","304.8","304.9","304.91","304.92","304.93","305.2","305.21","305.22","305.23","305.33","305.4","305.41","305.42","305.51","305.6","305.61","305.62","305.63","305.7","305.73","305.8","305.9","305.91","305.92","305.93","292.1","292.8","292.81","292.81","292.81","304","304.6","304.62","304.70","304.71","304.72","304.73","304.80","304.81","304.82","304.83","304.90","304.91","304.92","304.93","648.3","648.30","648.31","648.32","648.33","648.34","94.25","94.45","94.65","94.66","94.67","94.68","94.69","V87.4","304.20","304.21","304.22","304.23","305.60","970.81","F12.10","F12.11","F12.120","F12.19","F12.20","F12.21","F12.90","F13.10","F13.20","F13.21","F13.239","F14.10","F14.20","F14.21","F15.10","F15.11","F15.20","F15.21","F15.920","F15.93","F15.959","F16.11","F18.10","F19.10","F19.20","F19.21","F19.230","F19.90","F19.930","F19.939","F19.94","F19.950","F19.951","F19.982","F19.99","F12.121","F12.122","F12.129","F12.150","F12.151","F12.159","F12.180","F12.188","F12.220","F12.221","F12.222","F12.229","F12.250","F12.251","F12.259","F12.280","F12.288","F12.29","F12.920","F12.921","F12.922","F12.929","F12.950","F12.951"," F12.959","F12.980","F12.988","F12.988","F12.99","F14.120","F14.121","F14.122"," F14.129","F14.14","F14.150","F14.151","F14.159","F14.180","F14.181","F14.182","F14.188","F14.19"," F14.220","F14.221","F14.222","F14.229","F14.23"," F14.250","F14.251","F14.259","F14.280","F14.281","F14.282","F14.288","F14.29","F14.90","F14.920","F14.921","F14.922","F14.929","F14.94","F14.950","F14.959","F14.980","F14.981","F14.982","F14.988","F14.99","F15","F16","F18","F19","O35.5XX0","O35.5XX1","O35.5XX2","O35.5XX3","O35.5XX4","O35.5XX5","O35.5XX9","O99.320","O99.321","O99.322","O99.323","O99.324","O99.325","T40.5X2A","T40.5X2D","T40.5X2S","T40.7X1A","T40.7X1D","T40.7X1S","T40.7X2A","T40.7X2D","T40.7X2S","T40.7X3A","T40.7X3D","T40.7X3S","T40.7X4A","T40.7X4D","T40.7X4S","T40.7X5A","T40.7X5D","T40.7X5S","T40.7X6A","T40.7X6D","T40.7X6S","Z71.51"]
anxiety_icd = ["300","300.01","300.02","300.09","293.84","300.2","300.21","300.22","300.23","300.29","300.5","308","308.3","308.4","308.9","F41.9","F41.0","F41.1","F41.3","F41.8"]
depression_icd = ["296.3","296.31","296.32","296.33","296.34","298","296.35","296.36","296.99","311","293.83","296.2","296.21","296.22","296.23","296.24","296.25","296.26","296.3","296.31","296.32","296.33","296.34","296.35","296.36","300.4","F33.40","F33.9","F33.0","F33.1","F33.2","F33.3","F33.41","F33.42","F33.8","F32.9","F32.0","F32.1","F32.2","F32.3","F32.4","F32.5","F32.8","F32.81","F32.89","F33.9"]
suicide_icd = ["E950","E950.0","E950.1","E950.2","E950.3","E950.4","E950.5","E950.6","E950.7","E950.8","E950.9","E951","E951.0","E951.1","E951.8","E952","E952.0","E952.1","E952.8","E952.9","E953","E953.0","E953.1","E953.8","E953.9","E954","E955","E955.0","E955.1","E955.2","E955.3","E955.4","E955.5","E955.6","E955.7","E955.9","E956","E957","E957.0","E957.1","E957.2","E957.9","E958","E958.0","E958.1","E958.2","E958.3","E958.4","E958.5","E958.6","E958.7","E958.8","E958.9","E959","965","965.0","965.00","965.01","965.02","965.09","965.1","965.4","965.5","965.6","965.61","965.69","965.7","965.8","965.9","967","967.0","967.1","967.2","967.3","967.4","967.5","967.6","967.8","967.9","969","969.0","969.00","969.01","969.02","969.03","969.04","969.05","969.09","969.1","969.2","969.3","969.4","969.5","969.6","969.7","969.7","969.71","969.72","969.73","969.79","969.8","969.9","881","881.0","881.00","881.01","881.02","881.1","881.1","881.11","881.12","881.2","881.2","881.21","881.22","T14.91","T14.91XA","T14.91XD","T14.91XS","T36.0X2","T36.0X2A","T36.0X2D","T36.0X2S","T36.1X2","T36.1X2A","T36.1X2D","T36.1X2S","T36.2X2","T36.2X2A","T36.2X2D","T36.2X2S","T36.3X2","T36.3X2A","T36.3X2D","T36.3X2S","T36.4X2","T36.4X2A","T36.4X2D","T36.4X2S","T36.5X2","T36.5X2A","T36.5X2D","T36.5X2S","T36.6X2","T36.6X2A","T36.6X2D","T36.6X2S","T36.7X2","T36.7X2A","T36.7X2D","T36.7X2S","T36.8X2","T36.8X2A","T36.8X2D","T36.8X2S","T36.92","T36.92XA","T36.92XD","T36.92XS","T37.0X2","T37.0X2A","T37.0X2D","T37.0X2S","T37.1X2","T37.1X2A","T37.1X2D","T37.1X2S","T37.2X2","T37.2X2A","T37.2X2D","T37.2X2S","T37.3X2","T37.3X2A","T37.3X2D","T37.3X2S","T37.4X2","T37.4X2A","T37.4X2D","T37.4X2S","T37.5X2","T37.5X2A","T37.5X2D","T37.5X2S","T37.8X2","T37.8X2A","T37.8X2D","T37.8X2S","T37.92","T37.92XA","T37.92XD","T37.92XS","T38.0X2","T38.0X2A","T38.0X2D","T38.0X2S","T38.1X2","T38.1X2A","T38.1X2D","T38.1X2S","T38.2X2","T38.2X2A","T38.2X2D","T38.2X2S","T38.3X2","T38.3X2A","T38.3X2D","T38.3X2S","T38.4X2","T38.4X2A","T38.4X2D","T38.4X2S","T38.5X2","T38.5X2A","T38.5X2D","T38.5X2S","X71","X72","X73","X75","X76","X77","X78","X79","X80","X82","X83","X84","Y87.0","Z91.5"]
schizophrenia_icd = ["295","295.0","295.00","295.01","295.02","295.03","295.04","295.05","295.1","295.11","295.10","295.12","295.13","295.14","295.15","295.2","295.20","295.21","295.22","295.23","295.24","295.25","295.3","295.30","295.31","295.32","295.33","295.34","295.35","295.4","295.41","295.42","295.43","295.44","295.45","295.5","295.50","295.51","295.52","295.53","295.54","295.55","295.6","295.61","295.62","295.63","295.64","295.65","295.7","295.71","295.72","295.73","295.74","295.75","295.8","295.81","295.82","295.83","295.84","295.85","295.9","295.91","295.92","295.93","295.94","295.95","V11.0","F20.0","F20.89","F20.1","F20.2","F20.3","F20.5","F20.81","F20.9"]
bipolar_icd = ["296.0","296.01","296.02","296.03","296.04","296.05","296.06","296.1","296.11","296.12","296.13","296.14","296.15","296.16","296.4","296.41","296.42","296.43","296.44","296.45","296.46","296.5","296.51","296.52","296.53","296.54","296.55","296.56","296.6","296.61","296.62","296.63","296.64","296.65","296.66","296.7","296.8","296.81","296.82","296.89","F31","F31.1","F31.10","F31.11","F31.12","F31.13","F31.2","F31.3","F31.30","F31.31","F31.32","F31.4","F31.5","F31.6","F31.60","F31.61","F31.62","F31.63","F31.64","F31.7","F31.70 ","F31.71","F31.72","F31.73","F31.74","F31.75","F31.76","F31.77","F31.78","F31.8","F31.81","F31.89","F31.9"]
pain_icd = ["138","274","307.81","322.9","336","337.21","337.22","338.4","349","349.2","350.1","350.2","353","353.5","353.6","354","354.1","354.2","354.3","354.4","355.1","355.2","355.5","355.6","355.71","355.9","356.9","524.6","710","710.3","710.4","714","715.11","715.12","715.13","715.14","715.15","715.16","715.91","717.3","717.4","717.7","717.89","719.41","719.42","719.43","719.44","719.45","719.46","719.47","719.7","720","720.2","721","721.1","721.2","721.3","721.41","721.42","721.6","722","722.1","722.11","722.4","722.51","722.52","722.71","722.72","722.81","722.82","722.83","723","723.1","723.4","723.5","723.8","723.9","724.01","724.02","724.03","724.1","724.3","724.4","724.6","724.79","725","726","726.1","726.12","726.19","726.2","726.31","726.32","726.33","726.5","726.61","726.64","726.65","726.7","726.71","726.9","727.03","727.04","727.61","728.71","728.85","728.89","729.1","729.2","733","733.02","733.13","733.5","733.6","736.05","737.3","738.4","739.8","756.11","756.15","781","781.2","782","784","805.2","805.4","840.4","840.7","842.19","843.8","843.9","846","847","847.1","847.2","848.3","848.41","953.5","956.3","958.8","53.10 or 53.19","322.9","339","307.81","350.1","350.2","353","353.4","353.5","353.6","354","354","354.1","354.2","354.2","354.3","354.3","354.4","354.4","355","355","355.1","355.1","355.5","355.5","355.6","355.6","355.71","355.71","356.9","357.6","338.4","337.21","337.21","337.21","337.22","337.22","337.22","336","349","530.81","564.01","564.09","714","274","716.9","715.13","715.13","715.14","715.14","736.05","736.05","717.7","717.7","719.41","719.41","719.45","719.45","719.46","719.46","719.47","719.47","524.6","710","725","738.4","738.4","738.4","738.4","738.4","723.5","720","720","720.2","721.41","721","721","721.2","721.3","721.3","723","724.01","724.02","721.6","733.13","V54.27","733.13","733.13","V54.27","722.91","722","722","722.4","722.73","722.92","722.93","722.93","722.11","722.1","722.1","722.51","722.52","722.52","724.6 or 724.79","723.6","723.4","723.4","724.4","724.4","724.4","723.1","724.2","724.1","723.8","728.2","728.87","724.8","728.85","727.03","727.04","726.33","726.33","726.65 or 727.2","726.65 or 727.2","726.6","726.6","726.5","726.5","726.5","726.5","728.71","726","726","726.12","726.12","726.2","726.2","726.1","726.1","726.19 or 726.2","726.19 or 726.2","726.5","726.5","726.5","726.71","726.71","726.31","726.31","726.32","726.32","726.7","726.7","729.1","729.2","729.5","729.5","729.5","729.5","729.1","733","733.6","722.81","724.02","724.02","756.12","756.15","782","782","782","719.7","781.2","784","782.3","782.3","847","V58.89","847","V58.89","905.7","847.1","V58.89","848.3","847.2","V58.89","846.1","V58.89","840.7","956.3","956.3","B91 or G14","M10.00","G44.209","G03.9","G95.0","G90.519","G90.529","G89.4","G97.1","G96.12 or G96.19","G50.0","G50.1","G54.0","G54.5","G54.6","G56.00","G56.10","G56.20","G56.30","G56.40","G57.10","G57.20","G57.50","G57.60","G57.70","G58.9","G60.9","M26.601 or M26.602 or M26.603 or M26.609 or M26.69","M32.10","M33.03 or M33.13 or M33.90 or M33.93","M33.20","M06.9","M19.019","M19.029","M19.039","M19.049","M16.10","M17.10","M19.90","M23.239 or M23.305 or M23.339","M23.269","M22.40","M23.8x9","M25.519","M25.529","M25.539","M25.541 or M25.542 or M25.549 or M79.646","M25.559","M25.569","M25.579","R26.2","M45.9","M46.1","M47.812","M47.12","M47.814","M47.817","M47.14","M47.16","M48.10","M50.20","M51.26 or M51.27","M51.24 or M51.25","M50.30","M51.34 or M51.35","M51.36 or M51.37","M50.00","M51.04 or M51.05","M96.1","M96.1","M96.1","M48.02","M54.2","M54.12 or M54.13","M43.6","M53.82","M53.82","M48.04","M48.061","M48.062","M54.6","M54.30","M54.14 or M54.15 or M54.16 or M54.17","M43.27 or M43.28 or M53.2x7 or M53.3","M53.3","M35.3","M75.00","M75.100 or M75.50","M75.2","M75.80","M75.30 or M75.40 or M75.80","M77.00","M77.10","M70.20","M70.60 or M70.70 or M76.10 or M76.20","M76.899","M76.50","M70.40","M76.899 or M77.40","M76.60","M77.9","M65.30","M65.4","M75.120","M72.2","M62.40 or M62.838","M62.89","M60.9 or M79.1 or M79.7","M54.10 or M79.2","M81.0","M81.8","M48.50XA or M80.08XA or M84.48XA or M84.68XA","M85.30","M94.0","M21.339","M41.20","M43.00 or M43.10","M99.08","Q76.2","Q76.49","R25.0 or R25.1 or R25.2 or R25.3 or R25.9","R26.0 or R26.1 or R26.89 or R26.9","R20.0 or R20.1 or R20.2 or R20.3 or R20.8 or R20.9","R51 or G44.1","S22.009A","S32.009A","S43.429A","S43.439A","S63.8X9A","S73.199A","S73.109A or S76.919A","S33.8XXA","S13.4XXA or S13.8XXA","S23.3XXA or S23.8XXA","S33.5XXA","S23.41XA","S23.420A","S34.4XXA","S84.10XA","T79.8XXA","B02.29","G03.9","G44.009","G44.209","G50.0","G50.1","G54.0","G54.4","G54.5","G54.6","G56.01","G56.02","G56.12","G56.21","G56.22","G56.31","G56.32","G56.41","G56.42","G57.01","G57.02","G57.11","G57.12","G57.51","G57.52","G57.61","G57.62","G57.71","G57.72","G60.9","G62.0","G89.4","G90.511","G90.512","G90.513","G90.521","G90.522","G90.523","G95.0","G97.1","K21.9","K59.01","K59.09","M06.9","M10.00","M","M19.031","M19.032","M19.041","M19.042","M21.331","M21.332","M22.41","M22.42","M25.511","M25.512","M25.551","M25.552","M25.561","M25.562","M25.571","M25.572","M26.60","M32.9","M35.3","M43.06","M43.12","M43.13","M43.16","M43.17","M43.6","M45.5","M45.6","M46.1","M47.14","M47.22","M47.812","M47.814","M47.816","M47.817","M48.02","M48.04","M48.06","M48.07","M48.10","M48.54XA","M48.54XD","M48.55XA","M48.56XA","M48.56XD","M50.11","M50.12","M50.21","M50.22","M50.23","M50.31","M50.32","M51.06","M51.14","M51.16","M51.17","M51.24","M51.26","M51.27","M51.34","M51.36","M51.37","M53.3","M54.02","M54.12","M54.13","M54.14","M54.16","M54.17","M54.2","M54.5","M54.6","M54.81","M62.50","M62.81","M62.830","M62.838","M65.30","M65.4","M70.21","M70.22","M70.41","M70.42","M70.51","M70.52","M70.61","M70.62","M70.71","M70.72","M72.2","M75.01","M75.02","M75.21","M75.22","M75.41","M75.42","M75.51","M75.52","M75.81","M75.82","M76.01","M76.31","M76.32","M76.61","M76.62","M77.01","M77.02","M77.11","M77.12","M77.41","M77.42","M79.1","M79.2","M79.601","M79.602","M79.604","M79.605","M79.7","M81.0","M94.0","M96.1","M99.53","M99.73","Q76.2","Q76.49","R20.1","R20.2","R20.3","R26.2","R26.81","R51","R60.0","R60.1","S13.4XXA","S13.4XXD","S16.1XXA","S16.1XXD","S16.1XXS","S23.3XXA","S23.3XXD","S23.41XA","S33.5XXA","S33.5XXD","S33.6XXA","S33.6XXD","S43.432A","S84.11XA","S84.12XA"]
hepc_icd=["70.41","70.44","70.51","70.54","70.7","70.71","V02.60","V02.62","V02.69","B17.1","B17.10","B17.11","B18.2","B19.2","B19.20","B19.21","O98.4","O98.41","O98.411","O98.412","O98.413","O98.419","O98.42","O98.43"]
hiv_icd = ["42","42.1","42.2","43","43.1","43.2","43.3","44","44.9","79.53","795.71","V08","B20.0","B20.1","B20.2","B20.3","B20.4","B20.6","B20.7","B20.8","B21.0","B21.1","B21.2","B21.3","B21.7","B21.8","B22.0","B22.1","B22.2","B22.7","B23.0","B23.2","B23.8","B24","B97.35","F02.4","O98.7","O98.71","O98.711","O98.712","O98.713","O98.719","O98.72","O98.73","R75","Z11.4","Z21"]



In [17]:
def generate_column(colname,subset,dic):
    #total
    n = len(subset)
    #OUD from 1 ICD code
    oud1 = len(select_icd(oud_icd,subset,1))
    #OUD from 2+ ICD codes
    oud2 = len(select_icd(oud_icd,subset,2))
    #Average # of bouts
    temp = bout[bout.index.isin(subset)]
    avg_bout = round(temp.apply(lambda x:len(x)).mean(),2)
    #Average length of bouts (days)
    avg_bout_length = temp.explode().mean()
    #Average age at first prescription (years)
    first_p =df[df['GRID'].isin(subset)].groupby('GRID')['MED_DATE'].min()
    dob = covariate[covariate['GRID'].isin(subset)].set_index('GRID')['DOB']
    avg_age_fp = ((first_p-dob)/dt.timedelta(365)).mean()
    #Average age at first OUD ICD code (years)   
    first_oud =icd[icd['GRID'].isin(subset) & icd['ICD_CODE'].isin(oud_icd)].groupby('GRID')['ICD_DATE'].min()
    dob = covariate[covariate['GRID'].isin(subset)].set_index('GRID')['DOB']
    avg_age_oud_icd = ((first_oud-dob)/dt.timedelta(365)).mean()    
    #ICD code Z79.899
    Z79899 = len(select_icd(Z79899_icd,subset,1))
    #Alcohol Use Disorder 2 ICD
    aud2 = len(select_icd(aud_icd,subset,2))
    #Tobacco use Disorder 2 ICD
    tud2 = len(select_icd(tud_icd,subset,2))
    #Tobacco use Disorder 2 ICD
    cud2 = len(select_icd(cud_icd,subset,2))
    #Other Substance Abuse 2 ICD
    other_sud2 = len(select_icd(other_sud_icd,subset,2))
    #Anxiety 2 ICD
    anxiety2 = len(select_icd(anxiety_icd,subset,2))
    #Depression 2 ICD
    depress2 = len(select_icd(depression_icd,subset,2))
    #Suicide 2 ICD
    suicide2 = len(select_icd(suicide_icd,subset,2))
    #Schizophrenia 2 ICD
    schizophrenia2 = len(select_icd(schizophrenia_icd,subset,2))
    #bipolar 2 ICD
    bipolar2 = len(select_icd(bipolar_icd,subset,2))
    temp = icd[(icd['ICD_CODE'].isin(pain_icd)) & (icd['GRID'].isin(subset))]
    #Pain 2 ICD
    pain2 = len(select_icd(pain_icd,subset,2))
    #Average # of pain ICD codes
    if len(subset)>0:
        avg_pain = round(temp.groupby('GRID')['ICD_CODE'].count().sum()/len(subset))
    else:
        avg_pain=0
    #Average # of different pain conditions
    if len(subset)>0:
        avg_diff_pain = round(temp.groupby('GRID')['ICD_CODE'].nunique().sum()/len(subset))
    else:
        avg_diff_pain=0
    #Hepatitis C 2 ICD
    hepc2 = len(select_icd(hepc_icd,subset,2))
    #HIV 2 ICD
    hiv2 = len(select_icd(hiv_icd,subset,2))
    #Male %
    temp = covariate[covariate['GRID'].isin(subset)]
    male = round(temp['is.male'].mean()*100, 2)
    #Age,SD
    age = '{0:.2f} ({1:.2f})'.format(temp['age'].mean(),temp['age'].std())
    #Age <35 (%)
    if temp['GRID'].nunique()>0:
        age_35 = round(temp[temp['age']<35]['GRID'].nunique()/temp['GRID'].nunique()*100,2)
    else:
        age_35 = np.nan
    #White %
    white = round(temp['W'].mean()*100,2)
    #Asian %
    asian = round(temp['A'].mean()*100, 2)
    #African American %
    african = round(temp['B'].mean()*100,2)
    #Hispanic %
    hispanic = round((temp['ETHNICITY']=='HL').mean()*100, 2)
    #Other %
    other = round(((temp['W']+temp['A']+temp['B']+(temp['ETHNICITY']=='HL'))==0).mean()*100,2)
    #Average length of record (days)
    avg_len_record=round(temp['length_of_record'].mean(),2)
    #Average density of records
    avg_den_record=round(temp['density_of_record'].mean(),2)
    del temp

    def tostr(num):
        if n>0:
            return '{0} ({1:.2f}%)'.format(num,num/n*100)
        else:
            return 0
    ls = [n,tostr(oud1),tostr(oud2),avg_bout,str(avg_bout_length),avg_age_fp,avg_age_oud_icd,tostr(Z79899), \
          tostr(aud2), tostr(tud2),tostr(cud2), tostr(other_sud2), tostr(anxiety2), tostr(depress2), tostr(suicide2), tostr(schizophrenia2),tostr(bipolar2), \
          tostr(pain2),avg_pain, avg_diff_pain , tostr(hepc2), tostr(hiv2), male,age,age_35,white,asian,african,hispanic,other,avg_len_record,avg_den_record]
    dic[colname]=ls
    index = ["total","OUD from 1 ICD code","OUD from 2+ ICD codes","Average # of bouts","Average length of bouts (days)","Average age at first prescription (years)", "Average age at first OUD ICD code (years)", "ICD code Z79.899 (other long term drug therapy)", "Alcohol Use Disorder ", "Tobacco use Disorder", "Cannabis Use Disorder","Other Substance Abuse","Anxiety", "Depression", "Suicide","Schizophrenia", "Bipolar Disorder","Pain", "Average # of pain ICD codes","Average # of different pain conditions","Hepatitis C", "HIV", "Male %","Age,SD","Age <35 (%)","White %","Asian %","African American %","Hispanic %","Other %","Average length of record (days)","Average density of records"]
    for i in range(len(index)):
        print(colname,index[i],ls[i])

In [18]:
ls = [everyone_ls,no_prescription_ls, exposed_ls, oud1_ls, chronic2_ls]
ls_names=['full SD','no prescription', 'exposed controls', 'OUD from 1 ICD code', 'chronic2']
dic = {}
for i in range(len(ls)):
    generate_column(ls_names[i],ls[i],dic)


full SD total 549829
full SD OUD from 1 ICD code 6897 (1.25%)
full SD OUD from 2+ ICD codes 4215 (0.77%)
full SD Average # of bouts 1.77
full SD Average length of bouts (days) 81 days 18:56:56.572048
full SD Average age at first prescription (years) 42.16939210589833
full SD Average age at first OUD ICD code (years) 40.00039842615716
full SD ICD code Z79.899 (other long term drug therapy) 45714 (8.31%)
full SD Alcohol Use Disorder  9734 (1.77%)
full SD Tobacco use Disorder 29300 (5.33%)
full SD Cannabis Use Disorder 3505 (0.64%)
full SD Other Substance Abuse 9611 (1.75%)
full SD Anxiety 23394 (4.25%)
full SD Depression 41004 (7.46%)
full SD Suicide 4253 (0.77%)
full SD Schizophrenia 1511 (0.27%)
full SD Bipolar Disorder 7971 (1.45%)
full SD Pain 237673 (43.23%)
full SD Average # of pain ICD codes 7.0
full SD Average # of different pain conditions 2.0
full SD Hepatitis C 2210 (0.40%)
full SD HIV 1628 (0.30%)
full SD Male % 41.05
full SD Age,SD 51.12 (20.84)
full SD Age <35 (%) 27.94
ful

In [19]:
index = ["total","OUD from 1 ICD code","OUD from 2+ ICD codes","Average # of bouts","Average length of bouts (days)","Average age at first prescription (years)", "Average age at first OUD ICD code (years)","ICD code Z79.899","Alcohol Use Disorder", "Tobacco use Disorder", "Cannabis use Disorder", "Other Substance Abuse", "Anxiety", "Depression", "Suicide", "Schizophrenia", "bipolar", "Pain", "Average # of pain ICD codes","Average # of different pain conditions","Hepatitis C",  "HIV","Male %","Age,SD","Age <35 (%)","White %","Asian %","African American %","Hispanic %","Other %","Average length of record (days)","Average density of records"]
result = pd.DataFrame(dic,index)
result.to_csv('full-SD-opioid_2005-2017.csv')